In [1]:
from utilities import *
from core.initial import Insertions
from docplex.cp.model import *
from IPython.display import display


In [2]:
datafile = "D:/Google Drive/MSc MST-AUEB/_Thesis_/Main Thesis/Model Data.xlsx"
d = DataProvider(filepath=datafile, route='910')
model = CSPModel(d)
model.build_model()

model.data.head()

,initial_depot,final_depot,relief_point,time,trip_duration,start_time,end_time
trip,,,,,,,
0,Fix Station,Fix Station,NaN,05:00,40,300,340
1,Fix Station,Fix Station,NaN,05:20,40,320,360
2,Fix Station,Fix Station,NaN,05:55,40,355,395
3,Fix Station,Fix Station,NaN,06:20,44,380,424
4,Fix Station,Fix Station,NaN,06:40,44,400,444


In [ ]:
# initial = Insertions(model)
# initial.solve()

In [ ]:
# trips = [interval_var(start=trip.start_time,
#                       end=trip.end_time,
#                       size=trip.duration,
#                       name=f'Trip_{idx}') for idx, trip in enumerate(model.trips)]


In [3]:
ntrips = len(model.trips)
nduties = len(model.trips)

sub = CpoModel(name="Pricing_Subproblem")


min_start = model.data[start_time].min()
max_start = model.data[start_time].max()
min_end = model.data[end_time].min()
max_end = model.data[end_time].max()

duties = [interval_var(start=(min_start, max_start),
                       end=(min_end, max_end),
                       size=model.constraints.shift_span,
                       name=f"Duty_{i}",
                       optional=True)
          for i in range(nduties)]

trip2trip = integer_var_list(size=ntrips,
                             min=0,
                             max=ntrips + 1,
                             name='Trip2Trip')

trip2duty = integer_var_list(size=ntrips,
                             min=0,
                             max=nduties,
                             name='Trip2Duty')

start_times = [[integer_var(min=0,
                           max=model.constraints.shift_span,
                           name=f"StartTime-{i}-{j}")
               for j in range(nduties)] for i in range(ntrips)]

cdt = integer_var_list(size=nduties,
                       min=0,
                       max=model.constraints.continuous_driving,
                       name="CDT")

tdt = integer_var_list(size=nduties,
                       min=0,
                       max=model.constraints.total_driving,
                       name="TDT")

In [5]:
for i in range(ntrips):
    for j in range(ntrips):
        sub.add(sub.if_then(
            trip2trip[i] == j, model.end_times[i] <= model.start_times[j]))

for i in range(ntrips):
    for j in range(ntrips):
        sub.add(sub.if_then(
            trip2trip[i] == j, model.end_locs[i] <= model.start_locs[j]))

for i in range(ntrips):
    sub.add(trip2trip[i] != i)

for i in range(ntrips):
    for j in range(ntrips):
        sub.add(sub.if_then(trip2trip[i] == j, trip2duty[i] == trip2duty[j]))

In [6]:
def report_solution(cpsol: CpoSolveResult):
    trips_per_duty = {}
    for i in range(ntrips):
        _out = f"{i:>2} -> {cpsol[trip2trip[i]]} | Duty: {cpsol[trip2duty[i]]}"
        # print(_out)

        duty_id = cpsol[trip2duty[i]]

        if duty_id in trips_per_duty:
            trips_per_duty[duty_id].append(i)
        else:
            trips_per_duty[duty_id] = []
            trips_per_duty[duty_id].append(i)

    print(f'\n\nTotal Duties: {len(trips_per_duty.keys())}')

    for duty_id, duty_trips in trips_per_duty.items():
        df_trips = model.data.loc[duty_trips]

        span = df_trips[end_time].max() - df_trips[start_time].min()

        print(
            f'\n\n>>> Duty {duty_id} - Trips: {len(duty_trips)} - Drive Time: {df_trips[trip_duration].sum()} - Shift Span: {span}\n')
        display(df_trips)

In [7]:
msol = sub.solve()

report_solution(msol)

 ! --------------------------------------------------- CP Optimizer 20.1.0.0 --
 ! Satisfiability problem - 116 variables, 10150 constraints
 ! Initial process time : 0.08s (0.08s extraction + 0.00s propagation)
 !  . Log search space  : 600.2 (before), 600.2 (after)
 !  . Memory usage      : 2.5 MB (before), 2.5 MB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed    W       Branch decision
 *                     69  0.10s        1         4  = Trip2Duty_13
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 613
 ! Number of fails        : 10
 ! Total memory usage     : 20.6 MB (20.5 MB CP Optimizer + 0.0 MB Concert)
 ! Time spent in solve    : 0.10s (0.03s engine + 0.08s extraction)
 ! Search speed

,initial_depot,final_depot,relief_point,time,trip_duration,start_time,end_time
trip,,,,,,,
0,Fix Station,Fix Station,NaN,05:00,40,300,340
39,Fix Station,Fix Station,NaN,16:15,52,975,1027




>>> Duty 38 - Trips: 14 - Drive Time: 632 - Shift Span: 1140



,initial_depot,final_depot,relief_point,time,trip_duration,start_time,end_time
trip,,,,,,,
1,Fix Station,Fix Station,NaN,05:20,40,320,360
2,Fix Station,Fix Station,NaN,05:55,40,355,395
3,Fix Station,Fix Station,NaN,06:20,44,380,424
5,Fix Station,Fix Station,NaN,07:00,48,420,468
7,Fix Station,Fix Station,NaN,07:30,48,450,498
11,Fix Station,Fix Station,NaN,08:35,52,515,567
15,Fix Station,Fix Station,NaN,09:40,52,580,632
21,Fix Station,Fix Station,NaN,11:15,44,675,719
24,Fix Station,Fix Station,NaN,12:05,44,725,769




>>> Duty 39 - Trips: 7 - Drive Time: 316 - Shift Span: 945



...

,initial_depot,final_depot,relief_point,time,trip_duration,start_time,end_time
trip,,,,,,,
9,Fix Station,Fix Station,NaN,08:00,52,480,532
10,Fix Station,Fix Station,NaN,08:15,52,495,547
17,Fix Station,Fix Station,NaN,10:10,44,610,654
18,Fix Station,Fix Station,NaN,10:25,44,625,669
20,Fix Station,Fix Station,NaN,11:00,44,660,704
22,Fix Station,Fix Station,NaN,11:35,44,695,739
27,Fix Station,Fix Station,NaN,12:55,44,775,819
29,Fix Station,Fix Station,NaN,13:30,44,810,854
31,Fix Station,Fix Station,NaN,14:05,48,845,893




>>> Duty 46 - Trips: 2 - Drive Time: 100 - Shift Span: 408



,initial_depot,final_depot,relief_point,time,trip_duration,start_time,end_time
trip,,,,,,,
14,Fix Station,Fix Station,NaN,09:25,52,565,617
36,Fix Station,Fix Station,NaN,15:25,48,925,973




>>> Duty 51 - Trips: 1 - Drive Time: 48 - Shift Span: 48



,initial_depot,final_depot,relief_point,time,trip_duration,start_time,end_time
trip,,,,,,,
37,Fix Station,Fix Station,NaN,15:45,48,945,993


In [ ]:
type(msol[trip2duty[0]])

In [ ]:
trips = [8, 13, 30, 43, 46, 47, 49, 50, 52]

In [ ]:
model.data.loc[trips]

In [ ]:
for i in range(ntrips):
    sub.add(trip2trip[i] != 0)
    # sub.add(sum([trip2trip[j] == i for j in range(ntrips+1)]) == 1)

In [ ]:
for i in range(ntrips-1):
    for j in range(i + 1, ntrips):
        sub.add(end_before_start(trips[i], trips[j]))

In [ ]:
has_break = [[binary_var(name=f'Break_{i}_{j}') for j in range(nduties)] for i in range(ntrips)]
has_rest = [[binary_var(name=f'Rest_{i}_{j}') for j in range(nduties)] for i in range(ntrips)]

In [ ]:
trip_seq = sequence_var(trips)

In [ ]:
td_matrix = {}
for trip in model.trips:
    for duty in range(ntrips):
        td_matrix[(trip.ID, duty)] = binary_var(name=f"T{trip.ID}-D{duty}")

In [ ]:
end_of(trips[0]).equals(trips[1])

In [ ]:
INTERVAL_MAX